In [2]:
import cv2
import numpy as np
import mediapipe as mp
from math import hypot
import screen_brightness_control as sbc 
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
from comtypes import HRESULT, CLSCTX_ALL
from ctypes import cast ,POINTER

ImportError: cannot import name 'HRESULT' from 'ctypes' (/opt/anaconda3/lib/python3.11/ctypes/__init__.py)

In [2]:
import cv2
import numpy as np
import mediapipe as mp
from math import hypot
import os
import subprocess
from AppKit import NSScreen

def main():
    mpHands = mp.solutions.hands
    hands = mpHands.Hands(
        static_image_mode=False,
        max_num_hands=2,
        min_detection_confidence=0.75,
        min_tracking_confidence=0.75,
    )
    
    mpDraw = mp.solutions.drawing_utils
    
    cap = cv2.VideoCapture(0)
    
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    cap.set(cv2.CAP_PROP_FPS, 30)
    
    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            frame = cv2.flip(frame, 1)
            frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            processed = hands.process(frameRGB)
            left_landmark_list, right_landmark_list = get_left_right_landmarks(frame, processed, mpDraw, mpHands)
            
            if left_landmark_list:
                left_distance = get_distance(frame, left_landmark_list)
                if left_distance is not None:
                    b_level = np.interp(left_distance, [50, 220], [0, 100])
                    set_screen_brightness(b_level)
            
            if right_landmark_list:
                right_distance = get_distance(frame, right_landmark_list)
                if right_distance is not None:
                    vol_level = np.interp(right_distance, [50, 220], [0, 100])
                    set_system_volume(int(vol_level))
            
            cv2.imshow('Mouse', frame)
        
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
    finally:
        cap.release()
        cv2.destroyAllWindows()

def get_left_right_landmarks(frame, processed, mpDraw, mpHands):
    left_landmark_list = []
    right_landmark_list = []
    
    if processed.multi_hand_landmarks:
        for hand_landmarks, hand_handedness in zip(processed.multi_hand_landmarks, processed.multi_handedness):
            landmark_list = []
            for id, lm in enumerate(hand_landmarks.landmark):
                h, w, c = frame.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                if id == 4 or id == 8:  # Thumb tip or Index finger tip
                    landmark_list.append([cx, cy])
            if hand_handedness.classification[0].label == 'Left':
                left_landmark_list = landmark_list
            else:
                right_landmark_list = landmark_list
            mpDraw.draw_landmarks(frame, hand_landmarks, mpHands.HAND_CONNECTIONS)
    
    return left_landmark_list, right_landmark_list

def get_distance(frame, landmark_list):
    if len(landmark_list) < 2:
        return None
    
    (x1, y1), (x2, y2) = (landmark_list[0][0], landmark_list[0][1]), (landmark_list[1][0], landmark_list[1][1])
    
    cv2.circle(frame, (x1, y1), 7, (0, 0, 255), cv2.FILLED)
    cv2.circle(frame, (x2, y2), 7, (0, 0, 255), cv2.FILLED)
    cv2.line(frame, (x1, y1), (x2, y2), (0, 0, 255), 3)
    
    euclidean_distance = hypot(x2 - x1, y2 - y1)
    return euclidean_distance

def set_screen_brightness(brightness_level):
    # Use the 'brightness' command-line tool to set the screen brightness
    brightness_value = brightness_level / 100.0  # Convert to range [0.0, 1.0]
    os.system(f"brightness {brightness_value}")

def set_system_volume(volume_level):
    # Use AppleScript to set the system volume
    osascript_command = f"osascript -e 'set volume output volume {volume_level}'"
    os.system(osascript_command)

if __name__ == '__main__':
    main()


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2024-06-24 03:04:03.014 python[10895:245238] NSSoftLinking - The function 'SLSTransactionSetWindowTags' can't be found in the (null) framework.
2024-06-24 03:04:03.016 python[10895:245238] NSSoftLinking - The function 'SLSTransactionReorderWindows' can't be found in the (null) framework.
2024-06-24 03:04:03.017 python[10895:245238] NSSoftLinking - The function 'SLSTransactionOrderWindowGroupFrontConditionally' can't be found in the (null) framework.
2024-06-24 03:04:03.154 python[10895:245238] NSSoftLinking - The function 'SLSTransactionEnsureSpaceSwitchToActiveProcess' can't be found in the (null) framework.
